# Build Chatbot with Memory

Create conversations that remember context across messages.


In [ ]:
%pip install -qU pixeltable openai sentence-transformers spacy


In [ ]:
import os, getpass
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')


In [ ]:
import pixeltable as pxt
from pixeltable.functions import openai
from pixeltable.functions.huggingface import sentence_transformer
from pixeltable.iterators.string import StringSplitter


In [ ]:
# Step 1: Store conversation history
pxt.create_dir('chat', if_exists='ignore')
history = pxt.create_table('chat.messages', {'role': pxt.String, 'content': pxt.String}, if_exists='ignore')


In [ ]:
# Step 2: Create searchable memory
memory = pxt.create_view('chat.memory', history, if_exists='ignore',
    iterator=StringSplitter.create(text=history.content, separators='sentence'))
memory.add_embedding_index('text', if_exists='ignore',
    string_embed=sentence_transformer.using(model_id='intfloat/e5-large-v2'))


In [ ]:
# Step 3: Create context retrieval query
@pxt.query
def get_relevant_context(user_message: str):
    sim = memory.text.similarity(user_message)
    return memory.order_by(sim, asc=False).select(memory.text, sim=sim).limit(5)


In [ ]:
# Step 4: Create chat table with LLM
chat = pxt.create_table('chat.conversation', {'user_message': pxt.String}, if_exists='ignore')
chat.add_computed_column(context=get_relevant_context(chat.user_message), if_exists='ignore')

@pxt.udf
def build_prompt(context: list[dict], message: str) -> str:
    relevant = '\n'.join(item['text'] for item in context if item['sim'] > 0.3)
    return f"Context:\n{relevant}\n\nUser: {message}" if relevant else message

chat.add_computed_column(prompt=build_prompt(chat.context, chat.user_message), if_exists='ignore')
chat.add_computed_column(if_exists='ignore',
    response=openai.chat_completions(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': chat.prompt}]
    ).choices[0].message.content
)


In [ ]:
# Have a conversation
history.insert([{'role': 'user', 'content': 'My favorite color is blue'}])
chat.insert([{'user_message': 'What color do I like?'}])


In [ ]:
# View response
print(chat.select(chat.response).tail(1)['response'][0])


**What's Happening:**
- Conversation history stored in table
- Memory view splits messages into searchable sentences
- Semantic search finds relevant context
- LLM gets context + current message

**Variation:** Add conversation to history after each response:
```python
# After getting response
history.insert([
    {'role': 'user', 'content': user_message},
    {'role': 'assistant', 'content': response}
])
```

**Next:** `answer-questions-from-docs.ipynb` • `let-ai-search-web-for-answers.ipynb`
